In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
from quantfreedom.indicators.tv_indicators import rma_tv_2
from quantfreedom.plotting.plotting_base import plot_rsi

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [ ]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol="BTCUSDT",
    timeframe="1h",
    candles_to_dl=400,
)
close = candles[:, 4]

In [ ]:
candles_to_df(candles=candles)

In [ ]:
source = close
length = 14

In [ ]:
prev_source = np.roll(source,1)
prev_source[0] = np.nan

In [ ]:
change = source - prev_source

In [ ]:
gains = np.where(change > 0, change, 0)
losses = np.where(change < 0, -change, 0)

In [ ]:
gains[0] = np.nan
losses[0] = np.nan

In [ ]:
rma_gains, rma_losses = rma_tv_2(
    source_1=gains,
    source_2=losses,
    length=length,
)

In [ ]:
rs = rma_gains / rma_losses

In [ ]:
rs

In [ ]:
rsi = 100 - 100 / (1 + rs)

In [ ]:
rsi

In [ ]:
def tv_rsi(
    source: np.array,
    length: int,
):
    """
    Relative strength index https://www.tradingview.com/pine-script-reference/v5/#fun_ta.rsi
    """
    prev_source = np.roll(source, 1)
    prev_source[0] = np.nan
    change = source - prev_source

    gains = np.where(change > 0, change, 0)
    losses = np.where(change < 0, -(change), 0)
    gains[0] = np.nan
    losses[0] = np.nan

    rma_gains, rma_losses = rma_tv_2(
        source_1=gains,
        source_2=losses,
        length=length,
    )

    rs = rma_gains / rma_losses

    rsi = 100 - (100 / (1 + rs))
    return rsi

In [ ]:
tv_rsi(source=close, length=14)

In [ ]:
plot_rsi(candles=candles, indicator=rsi, ind_color="red")